# 图像卷积

上节我们解析了卷积层的原理，现在我们看看它的实际应用。由于卷积神经网络的设计是用于探索图像数据，本节我们将以图像为例。

## 互相关运算

严格来说，卷积层是个错误的叫法，因为它所表达的运算其实是*互相关运算*（cross-correlation），而不是卷积运算。
根据 [从全连接层到卷积](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/05.convolutional-neural-networks/02.why-conv.ipynb)中的描述，在卷积层中，输入张量和核张量通过(**互相关运算**)产生输出张量。

首先，我们暂时忽略通道（第三维）这一情况，看看如何处理二维图像数据和隐藏表示。在下图中，输入是高度为$3$、宽度为$3$的二维张量（即形状为$3 \times 3$）。卷积核的高度和宽度都是$2$，而卷积核窗口（或卷积窗口）的形状由内核的高度和宽度决定（即$2 \times 2$）。

![二维互相关运算。阴影部分是第一个输出元素，以及用于计算输出的输入张量元素和核张量元素：$0\times0+1\times1+3\times2+4\times3=19$.](../assets/correlation.svg)

在二维互相关运算中，卷积窗口从输入张量的左上角开始，从左到右、从上到下滑动。
当卷积窗口滑动到新一个位置时，包含在该窗口中的部分张量与卷积核张量进行按元素相乘，得到的张量再求和得到一个单一的标量值，由此我们得出了这一位置的输出张量值。
在如上例子中，输出张量的四个元素由二维互相关运算得到，这个输出高度为$2$、宽度为$2$，如下所示：

$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.
$$

注意，输出大小略小于输入大小。这是因为卷积核的宽度和高度大于1，
而卷积核只与图像中每个大小完全适合的位置进行互相关运算。
所以，输出大小等于输入大小$n_h \times n_w$减去卷积核大小$k_h \times k_w$，即：

$$(n_h-k_h+1) \times (n_w-k_w+1).$$

这是因为我们需要足够的空间在图像上“移动”卷积核。稍后，我们将看到如何通过在图像边界周围填充零来保证有足够的空间移动卷积核，从而保持输出大小不变。
接下来，我们在`corr2d`函数中实现如上过程，该函数接受输入张量`X`和卷积核张量`K`，并返回输出张量`Y`。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def corr2d(X, k):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

通过上图的输入张量`X`和卷积核张量`K`，我们来[**验证上述二维互相关运算的输出**]。

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 卷积层

卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。
所以，卷积层中的两个被训练的参数是卷积核权重和标量偏置。
就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。

基于上面定义的`corr2d`函数[**实现二维卷积层**]。在`__init__`构造函数中，将`weight`和`bias`声明为两个模型参数。前向传播函数调用`corr2d`函数并添加偏置。

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

高度和宽度分别为$h$和$w$的卷积核可以被称为$h \times w$卷积或$h \times w$卷积核。
我们也将带有$h \times w$卷积核的卷积层称为$h \times w$卷积层。

## 图像中目标的边缘检测

如下是[**卷积层的一个简单应用：**]通过找到像素变化的位置，来(**检测图像中不同颜色的边缘**)。
首先，我们构造一个$6\times 8$像素的黑白图像。中间四列为黑色（$0$），其余像素为白色（$1$）。

In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构造一个高度为$1$、宽度为$2$的卷积核`K`。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [6]:
K = torch.tensor([[1.0, -1.0]])

现在，我们对参数`X`（输入）和`K`（卷积核）执行互相关运算。
如下所示，[**输出`Y`中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘**]，其他情况的输出为$0$。

In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

现在我们将输入的二维图像转置，再进行如上的互相关运算。
其输出如下，之前检测到的垂直边缘消失了。
不出所料，这个[**卷积核`K`只可以检测垂直边缘**]，无法检测水平边缘。

In [8]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习卷积核

如果我们只需寻找黑白边缘，那么以上`[1, -1]`的边缘检测器足以。然而，当有了更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计滤波器。那么我们是否可以[**学习由`X`生成`Y`的卷积核**]呢？

现在让我们看看是否可以通过仅查看“输入-输出”对来学习由`X`生成`Y`的卷积核。
我们先构造一个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较`Y`与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

In [9]:
# 构造一个二维卷积层，它具有1个输出通道和形状为(1, 2)的卷积核

conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度）
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2 # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 4.966
epoch 4, loss 0.960
epoch 6, loss 0.213
epoch 8, loss 0.057
epoch 10, loss 0.018


在$10$次迭代之后，误差已经降到足够低。现在我们来看看我们[**所学的卷积核的权重张量**]。

In [10]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0016, -0.9765]])

细心的读者一定会发现，我们学习到的卷积核权重非常接近我们之前定义的卷积核`K`。

## 互相关和卷积

回想一下我们在[从全连接层到卷积](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/05.convolutional-neural-networks/02.why-conv.ipynb)中观察到的互相关和卷积运算之间的对应关系。
**为了得到正式的*卷积*运算输出，我们需要执行 :eqref:`eq_2d-conv-discrete`中定义的严格卷积运算，而不是互相关运算。**
幸运的是，它们差别不大，**我们只需水平和垂直翻转二维卷积核张量，然后对输入张量执行*互相关*运算**。

值得注意的是，由于卷积核是从数据中学习到的，因此无论这些层执行严格的卷积运算还是互相关运算，卷积层的输出都不会受到影响。
为了说明这一点，假设卷积层执行*互相关*运算并学习上图中的卷积核，该卷积核在这里由矩阵$\mathbf{K}$表示。
假设其他条件不变，当这个层执行严格的*卷积*时，学习的卷积核$\mathbf{K}'$在水平和垂直翻转之后将与$\mathbf{K}$相同。
也就是说，当卷积层对上图中的输入和$\mathbf{K}'$执行严格*卷积*运算时，将得到与互相关运算上图中相同的输出。

为了与深度学习文献中的标准术语保持一致，我们将继续把“互相关运算”称为卷积运算，尽管严格地说，它们略有不同。
此外，对于卷积核张量上的权重，我们称其为*元素*。

## 特征映射和感受野

如在[从全连接层到卷积一节讲的通道](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/05.convolutional-neural-networks/02.why-conv.ipynb)中所述， 上图中输出的卷积层有时被称为*特征映射*（feature map），因为它可以被视为一个输入映射到下一层的空间维度的转换器。
在卷积神经网络中，对于某一层的任意元素$x$，其*感受野*（receptive field）是指在前向传播期间可能影响$x$计算的所有元素（来自所有先前层）。

请注意，感受野可能大于输入的实际大小。让我们用上图为例来解释感受野：
给定$2 \times 2$卷积核，阴影输出元素值$19$的感受野是输入阴影部分的四个元素。
假设之前输出为$\mathbf{Y}$，其大小为$2 \times 2$，现在我们在其后附加一个卷积层，该卷积层以$\mathbf{Y}$为输入，输出单个元素$z$。
在这种情况下，$\mathbf{Y}$上的$z$的感受野包括$\mathbf{Y}$的所有四个元素，而输入的感受野包括最初所有九个输入元素。
因此，当一个特征图中的任意元素需要检测更广区域的输入特征时，我们可以构建一个更深的网络。

## 小结

* 二维卷积层的核心计算是二维互相关运算。最简单的形式是，对二维输入数据和卷积核执行互相关操作，然后添加一个偏置。
* 我们可以设计一个卷积核来检测图像的边缘。
* 我们可以从数据中学习卷积核的参数。
* 学习卷积核时，无论用严格卷积运算或互相关运算，卷积层的输出不会受太大影响。
* 当需要检测输入特征中更广区域时，我们可以构建一个更深的卷积网络。
* 卷积运算和互相关运算差别不大，我们只需要水平和垂直翻转二维卷积核张量，然后对输入张量执行互相关运算得到的输出就是严格的卷积运算的输出。

## 练习

1. 构建一个具有对角线边缘的图像`X`。
    1. 如果将本节中举例的卷积核`K`应用于`X`，会发生什么情况？
    1. 如果转置`X`会发生什么？
    1. 如果转置`K`会发生什么？
1. 在我们创建的`Conv2D`自动求导时，有什么错误消息？
1. 如何通过改变输入张量和卷积核张量，将互相关运算表示为矩阵乘法？
1. 手工设计一些卷积核。
    1. 二阶导数的核的形式是什么？
    1. 积分的核的形式是什么？
    1. 得到$d$次导数的最小核的大小是多少？

### 练习一

1. 构建一个具有对角线边缘的图像`X`。
   1. 如果将本节中举例的卷积核`K`应用于`X`，会发生什么情况？
    1. 如果转置`X`会发生什么？
    1. 如果转置`K`会发生什么？

**解答：**

**第1问：**

&emsp;&emsp;在对角线处分别为 $1$ 和 $-1$ 的数据，其他区域都为 $0$。

In [11]:
# 代码验证
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape # 获取卷积核的高度和宽度
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 初始化输出矩阵 Y，大小为 (X行数 - K行数 + 1) x (X列数 - K列数 + 1)
    for i in range(Y.shape[0]): # 遍历 Y 的列数
        for j in range(Y.shape[1]): # 遍历 Y 的列数
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # 在 Y 的每个位置，计算 X 与 K 的互相关运算结果
    return Y

# 创建一个8x8的单位矩阵 X
X = torch.eye(8)
print(X)
# 创建一个1x2的卷积核 K，该卷积核在X上进行互相关运算
K = torch.tensor([[1.0, -1.0]])
# 调用corr2d函数，计算互相关运算结果Y
Y = corr2d(X, K)
# 打印互相关运算结果Y
print(Y)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])
tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.]])


**第2问：**

&emsp;&emsp;`X`转置后，结果不变。

In [12]:
# 代码验证
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape # 获取卷积核的高度和宽度
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 初始化输出矩阵 Y，大小为 (X行数 - K行数 + 1) x (X列数 - K列数 + 1)
    for i in range(Y.shape[0]): # 遍历 Y 的列数
        for j in range(Y.shape[1]): # 遍历 Y 的列数
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # 在 Y 的每个位置，计算 X 与 K 的互相关运算结果
    return Y

# 创建一个8x8的单位矩阵 X
X = torch.eye(8)
print(X.T)
# 创建一个1x2的卷积核 K，该卷积核在X上进行互相关运算
K = torch.tensor([[1.0, -1.0]])
# 调用corr2d函数，计算互相关运算结果Y
Y = corr2d(X.T, K) # 在 X 的转置上应用卷积核 K，得到卷积响应图像 Y
# 打印互相关运算结果Y
print(Y)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])
tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.]])


**第3问：**

&emsp;&emsp;`K`转置后，结果也会发生转置。

In [13]:
# 代码验证
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape # 获取卷积核的高度和宽度
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 初始化输出矩阵 Y，大小为 (X行数 - K行数 + 1) x (X列数 - K列数 + 1)
    for i in range(Y.shape[0]): # 遍历 Y 的列数
        for j in range(Y.shape[1]): # 遍历 Y 的列数
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # 在 Y 的每个位置，计算 X 与 K 的互相关运算结果
    return Y

# 创建一个8x8的单位矩阵 X
X = torch.eye(8)
print(X)
# 创建一个1x2的卷积核 K，该卷积核在X上进行互相关运算
K = torch.tensor([[1.0, -1.0]])
print(K.T)
# 调用corr2d函数，计算互相关运算结果Y
Y = corr2d(X, K.T) # 在 X 应用卷积核 K 的转置，得到卷积响应图像 Y
# 打印互相关运算结果Y
print(Y)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])
tensor([[ 1.],
        [-1.]])
tensor([[ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -1.]])


### 练习二

2. 在我们创建的`Conv2D`自动求导时，有什么错误消息？

**解答：**

&emsp;&emsp;会提示维度不对称的错误信息，因为`torch`提供的二维卷积层是`nn.Conv2d()`采用的是四维输入和输出格式（批量大小、通道、高度、宽度），而我们自定义的仅仅是二维的。

&emsp;&emsp;使用`nn.Conv2d()`。

In [14]:
# 代码验证
import torch
from torch import nn
from d2l import torch as d2l

# 定义二维互相关运算函数
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape # 获取卷积核的高度和宽度
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 初始化输出矩阵 Y
    for i in range(Y.shape[0]): # 遍历输出矩阵的每一行
        for j in range(Y.shape[1]): # 遍历输出矩阵的每一列
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # 执行互相关运算并赋值给Y的相应位置
    return Y

# 定义一个二维卷积层类
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__() # 调用nn.Module的构造函数
        self.weight = nn.Parameter(torch.rand(kernel_size)) # 初始化卷积参数
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x): # 定义前向传播函数
        return corr2d(x, self.weight) + self.bias # 返回卷积运算结果

# 创建一个6x8的矩阵，中间4列为0
X = d2l.ones((6, 8))
X[:, 2:6] = 0

# 创建一个1x2的卷积核
K = d2l.tensor([[1.0, -1.0]])

# 进行卷积运算
Y = corr2d(X, K)

# 对X进行转置，然后再次进行卷积运算
corr2d(d2l.transpose(X), K)

# 创建一个卷积层，具有1个输出通道和（1, 2）形状的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)


# 调整X和Y的形状以符合PyTorch卷积层的输入输出要求
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

# 设置学习率
lr = 3e-2

# 尝试进行10次迭代
try:
    for i in range(10):
        Y_hat = conv2d(X) # 前向传播
        l = (Y_hat - Y) ** 2 # 计算损失
        conv2d.zero_grad() # 梯度消失
        l.sum().backward() # 反向传播
        conv2d.weight.data[:] -= lr * conv2d.weight.grad # 更新权重
        if (i + 1) % 2 == 0: # 每两个epoch打印一次损失值
            print(f'epoch {i+1}, loss {l.sum():.3f}')
except Exception as e: # 捕获并打印任何异常
    print(e)

epoch 2, loss 3.682
epoch 4, loss 0.925
epoch 6, loss 0.281
epoch 8, loss 0.099
epoch 10, loss 0.038


&emsp;&emsp;使用创建的`Conv2D`时可能会报如下错误。

In [15]:
# 代码验证
import torch
from torch import nn
from d2l import torch as d2l

# 定义二维互相关运算函数
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape # 获取卷积核的高度和宽度
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 初始化输出矩阵 Y
    for i in range(Y.shape[0]): # 遍历输出矩阵的每一行
        for j in range(Y.shape[1]): # 遍历输出矩阵的每一列
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # 执行互相关运算并赋值给Y的相应位置
    return Y

# 定义一个二维卷积层类
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__() # 调用nn.Module的构造函数
        self.weight = nn.Parameter(torch.rand(kernel_size)) # 初始化卷积参数
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x): # 定义前向传播函数
        return corr2d(x, self.weight) + self.bias # 返回卷积运算结果

# 创建一个6x8的矩阵，中间4列为0
X = d2l.ones((6, 8))
X[:, 2:6] = 0

# 创建一个1x2的卷积核
K = d2l.tensor([[1.0, -1.0]])

# 进行卷积运算
Y = corr2d(X, K)

# 对X进行转置，然后再次进行卷积运算
corr2d(d2l.transpose(X), K)

# 创建一个卷积层，具有1个输出通道和（1, 2）形状的卷积核
conv2d = Conv2D(kernel_size=(1, 2))


# 调整X和Y的形状以符合PyTorch卷积层的输入输出要求
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

# 设置学习率
lr = 3e-2

# 尝试进行10次迭代
try:
    for i in range(10):
        Y_hat = conv2d(X) # 前向传播
        l = (Y_hat - Y) ** 2 # 计算损失
        conv2d.zero_grad() # 梯度消失
        l.sum().backward() # 反向传播
        conv2d.weight.data[:] -= lr * conv2d.weight.grad # 更新权重
        if (i + 1) % 2 == 0: # 每两个epoch打印一次损失值
            print(f'epoch {i+1}, loss {l.sum():.3f}')
except Exception as e: # 捕获并打印任何异常
    print(e)

The size of tensor a (0) must match the size of tensor b (7) at non-singleton dimension 3


&emsp;&emsp;可以通过下述方式调整。参数 X 和 Y 去掉批次大小和通道。

In [16]:
# 代码验证
import torch
from torch import nn
from d2l import torch as d2l

# 定义二维互相关运算函数
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape # 获取卷积核的高度和宽度
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 初始化输出矩阵 Y
    for i in range(Y.shape[0]): # 遍历输出矩阵的每一行
        for j in range(Y.shape[1]): # 遍历输出矩阵的每一列
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # 执行互相关运算并赋值给Y的相应位置
    return Y

# 定义一个二维卷积层类
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__() # 调用超类的构造函数
        self.weight = nn.Parameter(torch.rand(kernel_size)) # 随机初始化卷积的权重
        self.bias = nn.Parameter(torch.zeros(1)) # 初始化偏置为0

    def forward(self, x): # 定义前向传播函数
        return corr2d(x, self.weight) + self.bias # 返回卷积运算结果

# 创建一个6x8的矩阵，中间4列为0
X = d2l.ones((6, 8))
X[:, 2:6] = 0

# 创建一个1x2的卷积核
K = d2l.tensor([[1.0, -1.0]])

# 进行卷积运算
Y = corr2d(X, K)

# 对X进行转置，然后再次进行卷积运算
corr2d(d2l.transpose(X), K)

# 创建一个卷积层，具有1个输出通道和（1, 2）形状的卷积核
conv2d = Conv2D(kernel_size=(1, 2))


# 调整X和Y的形状以符合PyTorch卷积层的输入输出要求
X = X.reshape((6, 8))
Y = Y.reshape((6, 7))

# 设置学习率
lr = 3e-2

# 尝试进行10次迭代
try:
    for i in range(10):
        Y_hat = conv2d(X) # 前向传播
        l = (Y_hat - Y) ** 2 # 计算损失
        conv2d.zero_grad() # 梯度消失
        l.sum().backward() # 反向传播
        conv2d.weight.data[:] -= lr * conv2d.weight.grad # 更新权重
        if (i + 1) % 2 == 0: # 每两个epoch打印一次损失值
            print(f'epoch {i+1}, loss {l.sum():.3f}')
except Exception as e: # 捕获并打印任何异常
    print(e)

epoch 2, loss 18.856
epoch 4, loss 6.629
epoch 6, loss 2.531
epoch 8, loss 1.006
epoch 10, loss 0.407


### 练习三

3. 如何通过改变输入张量和卷积核张量，将互相关运算表示为矩阵乘法？

In [17]:
# 代码验证
import torch
from torch import nn
from d2l import torch as d2l

# 定义一个函数来通过矩阵乘法实现二维卷积操作
def conv2d_by_mul(X, K):
    # 获取卷积核大小
    h, w = K.shape
    # 计算输出图像大小
    outh = X.shape[0] - h + 1
    outw = X.shape[1] - w + 1
    print(f'outh:{outh}')
    print(f'outw:{outw}')
    # 调整卷积核形状以便做乘法
    K = K.reshape(-1, 1)
    # 将输入图像切成卷积核大小的块打平成一维，存放在列表 Y 中
    Y = []
    for i in range(outh):
        for j in range(outw):
            Y.append(X[i:i + h, j:j + w].reshape(-1))
            print(X[i:i + h, j:j + w].reshape(-1))
        # 将列表 Y 转为张量，每行代表一块的打平结果
        Y = torch.stack(Y, 0)
        print(f'Y:torch.stack:{Y}')
        # 用矩阵乘法表示互相关运算
        print(f'K:{K}')
        res = (torch.matmul(Y, K)).reshape(outh, outw)
        # 返回输出结果
        return res

# 示例
# 创建一个2x3的输入图像 X其中第二列的元素被设置为0
X = torch.ones((2, 3))
X[:, 1] = 0 # 将第二列的所有元素设置为0
# 打印输入图像 X
print(f'原始：{X}')
# 创建一个2x3的卷积核 K，所有元素均为1
K = torch.ones((2, 3))
# 调用自定义的 conv2d_by_mul 函数来执行二维互相关运算
output = conv2d_by_mul(X, K)
# 打印互相关运算的结果
print(output)

原始：tensor([[1., 0., 1.],
        [1., 0., 1.]])
outh:1
outw:1
tensor([1., 0., 1., 1., 0., 1.])
Y:torch.stack:tensor([[1., 0., 1., 1., 0., 1.]])
K:tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])
tensor([[4.]])


### 练习四

4. 手工设计一些卷积核。
    1. 二阶导数的核的形式是什么？
    1. 积分的核的形式是什么？
    1. 得到$d$次导数的最小核的大小是多少？

**解答：**

**第1问：**

&emsp;&emsp;一维的二阶导数的核的形式是：

$$\begin{bmatrix}-1, & 2, & -1,\end{bmatrix}$$

&emsp;&emsp;这个一维卷积可以用于计算信号或函数在每个点的二阶导数。它通过对信号进行两次微分来强调信号的曲率和变化率，通常用于边缘检测和特征检测等任务。

&emsp;&emsp;在二维情况下，可以使用以下卷积核来计算图像的二阶导数：

\begin{bmatrix}
    0, &  1, & 0 \\
    1, & -4, & 1 \\
    0, &  1, & 0 \\
\end{bmatrix}

**第2问：**

&emsp;&emsp;积分的核的形式是：

$$\begin{bmatrix}1 & 1 & 1 & \cdots & 1\end{bmatrix}$$

**第3问：**

&emsp;&emsp;得到 `d` 次导数的最小核的大小是 $d+1$。例如，一阶导数的最小核大小为$2$，二阶导数的最小核大小为 $3$，三阶导数的最小核大小为 $4$，以此类推。